<a href="https://colab.research.google.com/github/jruiz1968/03MAIR---Algoritmos-de-Optimizacion---2019/blob/master/AG2/Julio_Ruiz_Gonzalez_AG2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
TARIFAS=[[0,5,4,3,999,999,999],
        [999,0,999,2,3,999,1],
        [999,999,0,1,999,4,10],
        [999,999,999,0,5,6,9],
        [999,999,999,999,0,999,4],
        [999,999,999,999,999,0,3],
        [999,999,999,999,999,999,0]]

In [20]:
def Precios (TARIFAS):
  
  N= len(TARIFAS[0])
  
  PRECIOS = [[9999]*N for i in [9999]*N ]
  RUTA=[[""]*N for i in [""]*N]
  
  for i in range(N-1):
      for j in range(i+1,N):
        MIN=TARIFAS[i][j]
        RUTA[i][j]=i
        
        for k in range (i,j):
          if PRECIOS[i][k] + TARIFAS[k][j] < MIN:
            MIN = PRECIOS[i][k] + TARIFAS[k][j]
            RUTA[i][j] = k
        PRECIOS[i][j] = MIN
  return PRECIOS, RUTA

PRECIOS,RUTA= Precios(TARIFAS)
print(PRECIOS)
print(RUTA)
#CALCULAR LA MEJOR RUTA

def calcular_ruta(RUTA, desde, hasta):
  if desde==hasta:
    return desde
  else:
    return str(calcular_ruta(RUTA, desde, RUTA[desde][hasta])) + '-' + str(RUTA[desde][hasta])
  
calcular_ruta(RUTA,1,5)
                                                                           


[[9999, 5, 4, 3, 8, 8, 6], [9999, 9999, 999, 2, 3, 8, 1], [9999, 9999, 9999, 1, 6, 4, 7], [9999, 9999, 9999, 9999, 5, 6, 9], [9999, 9999, 9999, 9999, 9999, 999, 4], [9999, 9999, 9999, 9999, 9999, 9999, 3], [9999, 9999, 9999, 9999, 9999, 9999, 9999]]
[['', 0, 0, 0, 1, 2, 1], ['', '', 1, 1, 1, 3, 1], ['', '', '', 2, 3, 2, 5], ['', '', '', '', 3, 3, 3], ['', '', '', '', '', 4, 4], ['', '', '', '', '', '', 5], ['', '', '', '', '', '', '']]


'1-1-3'

Asignacion de tareas. Ramificacion y poda

In [0]:
import itertools
from functools import wraps

def calcular_tiempo(f):
  @wraps(f)
  def cronometro(*args, **kwargs):
    t_inicial = time()
    salida= f(*args,**kwargs)
    t_final= time()
    print('Tiempo transcurrido ( en segundos): {}', format(t_final - t_inicial))
    return salida
  return cronometro

In [0]:
COSTES=[[11,12,18,40],
       [14,15,13,22],
       [11,17,19,23],
       [17,14,20,28]]

In [27]:
def valor(S,COSTES):
    VALOR=0
    for i in range(len(S)):
      VALOR+= COSTES[S[i]][i]
      
    return VALOR
  
valor ((0,1,2,3), COSTES)

73

POR FUERZA BRUTA

In [28]:
def fuerza_bruta(COSTES):
    mejor_valor=10e10
    mejor_solucion= ''
    
    for S in list(itertools.permutations(range(len(COSTES)))):
      valor_tmp = valor(S, COSTES)
      if valor_tmp < mejor_valor:
        mejor_solucion=S
        mejor_valor=valor_tmp
        
    print("La mejor solucion es :" ,mejor_solucion)
    
    
fuerza_bruta(COSTES)


La mejor solucion es : (0, 3, 1, 2)


In [29]:
# Funcion para estimar un coste inferior para una solucion parcial
#Coste inferior para soluciones parciales
#(1,3) Se asigna la tarea 1 al agente 0 y la tarea 3 al agente 1

def CI(S,COSTES):
  VALOR=0
  #Valores Establecidos
  for i in range(len(S)):
    VALOR+= COSTES[i][S[i]]
    
 #Estimacion
  for i in range( len(COSTES)):
    if i not in S:
      VALOR+= min([COSTES[j][i] for j in range(len(S), len(COSTES))])
  return VALOR    


CI((2,0), COSTES)

#(0,2)->(0,2,1),(0,2,3)

#{'S':(0,2), 'ci':34}

def crear_hijos(NODO, N):
  HIJOS = []
  for i in range(N):
    if i not in NODO:
      HIJOS:append({'S': NODO+(i,)})
        
  return HIJOS


crear_hijos((0,2), 4)

[]

In [33]:
def ramificacion_y_poda(COSTES):
  DIMENSION= len(COSTES)
  #(0,1,2,3)
  MEJOR_SOLUCION= tuple(i for i in range(DIMENSION))
  
  CotaSup= valor(MEJOR_SOLUCION,COSTES)
  
  NODOS=[]
  NODOS.append({'S':(), 'ci': CI((), COSTES) })
  
 
  
  iteracion=0
  
  while (len(NODOS)>0):
    iteracion+=1
    print('\n#', iteracion)
    
    nodo_prometedor = min(NODOS, key=lambda x:x['ci'])
    
    #Hijos
    
    HIJOS= [{'S':x['S'], 'ci': CI(x['S'], COSTES)} for x in crear_hijos(nodo_prometedor['S'], DIMENSION)]
    
    NODO_FINAL = [x for x in HIJOS if len(x['S']) ==DIMENSION]
    
    if len(NODO_FINAL)>0:
      if NODO_FINAL[0]['ci']< CotaSup:
        CotaSup=NODO_FINAL[0]['ci']
        MEJOR_SOLUCION=NODO_FINAL[0]
        
    #poda    
    HIJOS=[x for x in HIJOS if x['ci']<CotaSup]
    
    #Eliminamos el nodo expandido
    NODOS= [ x for x in NODOS if x['S'] != nodo_prometedor['S']]
    
    #Añadimos los hijos
    NODOS.extend(HIJOS)
    
    print(NODOS)
    
  print('La mejor solucion es:', MEJOR_SOLUCION)
  
  

ramificacion_y_poda(COSTES)
  


# 1
[]
La mejor solucion es: (0, 1, 2, 3)
